# Parameters

In [1]:
year = 2022
month = 'Marzo'
filaname_edi = '01.03.2022 MdP EDI.xlsx'
filaname_sap = '(2022_03_01@10.58hrs) MdP Clientes Nac SAP.xlsx'
filaname_amazon = 'Amazon MdP SaP 2022_02_08.xlsx'

# Libraries

In [2]:
from datetime import datetime
import pandas as pd

# Import data

In [3]:
months_number = {
    'Enero': '01',
    'Febrero': '02',
    'Marzo': '03',
    'Abril': '04',
    'Mayo': '05',
    'Junio': '06',
    'Julio': '07',
    'Agosto': '08',
    'Septiembre': '09',
    'Octubre': '10',
    'Noviembre': '11',
    'Diciembre': '12'
}

In [4]:
path_load = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\México\Precios\Data\{0}\{2} {1}\Raw'
path_load = path_load.format(year, month, months_number[month])

path_export = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\México\Precios\Data\{0}\{2} {1}\Output'
path_export = path_export.format(year, month, months_number[month])

## SAP

In [ ]:
%%time
df_sap = pd.read_excel(path_load + '\\' + filaname_sap, header=3, usecols='I, P, V:X, Z')

In [ ]:
df_sap.shape

In [ ]:
df_sap.rename({ 
    'Válido de':'Desde', 
    'CLIENTE':'Cliente',
    'CÓDIGO DE BARRAS':'ProPstCodBarras',
    'DESCRIPCIÓN':'Descripción',
    'PF SAP\n(PIEZA)':'Precio'
}, 
    axis=1, 
    inplace=True
)

In [ ]:
df_sap = df_sap[df_sap['Cliente']!='Amazon']
df_sap.reset_index(drop=True, inplace=True)

## EDI

In [ ]:
%%time
df_edi = pd.read_excel(path_load + '\\' + filaname_edi, header=5, usecols='A, E:H, L, O')

In [ ]:
df_edi = df_edi[df_edi['CLIENTE']==1100024]
df_edi.drop(['CLIENTE'], axis=1, inplace=True)
df_edi.reset_index(drop=True, inplace=True)

In [ ]:
df_edi.shape

In [ ]:
df_edi.rename({
    'NOMBRE CLIENTE':'Cliente',
    'CÓDIGO DE BARRAS':'ProPstCodBarras', 
    'MATERIAL':'Material', 
    'DESCRIPCIÓN':'Descripción',
    'Válido de':'Desde', 
    'PF ZD01':'Precio'
}, 
    axis=1, 
    inplace=True
)

# Amazon

In [ ]:
df_amazon = pd.read_excel(path_load + '\\' + filaname_amazon, usecols='B, I:J, P, V, Z', sheet_name='MdP')

In [ ]:
df_amazon.drop_duplicates(inplace=True)

In [ ]:
df_edi.columns

In [ ]:
df_amazon.columns

In [ ]:
df_amazon.rename({
    'COD BARRAS':'ProPstCodBarras',
    'Texto breve del material de cliente':'Descripción',
    'Válido de':'Desde', 
    'CLIENTE':'Cliente', 
    'PF SAP\n(PIEZA)':'Precio'
}, 
    axis=1, 
    inplace=True
)

# Export

In [ ]:
data = pd.concat([df_sap, df_edi, df_amazon])

data.drop_duplicates(inplace=True)

In [ ]:
filename_export = '{0} {1}_{2}.xlsx'
date_time_now = datetime.today().strftime('%Y%m%d_%H%M')
filename_export = filename_export.format(year, month, date_time_now)

data.to_excel(
    path_export + '\\' + filename_export,
    index=False,
    sheet_name='Precios MEX'
)